In [3]:
%pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 7.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 12.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488498 sha256=d8645f70ea55fb6a43fff8e2e2c7ff70db1222b55a23f7a57c6a66cb2f9a2598
  Stored in directory: /Users/shreyan/Library/Caches/pip/wheels/92/09/11/aa01d01a7f005fda8a66ad71d2be7f8aa341bddafb27eee3c7
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [1]:
%ls -all csv_folder/

total 4591464
drwxr-xr-x  37 shreyan  staff       1184 Jul  6 22:15 ./
drwxr-xr-x  13 shreyan  staff        416 Jul  7 16:59 ../
-rw-r--r--   1 shreyan  staff     463011 Jul  6 22:15 airportdb@airplane_type@@0.csv
-rw-r--r--   1 shreyan  staff     259868 Jul  6 22:13 airportdb@airport@@0.csv
-rw-r--r--   1 shreyan  staff     974612 Jul  6 22:15 airportdb@airport_geo@@0.csv
-rw-r--r--   1 shreyan  staff   68318660 Jul  6 22:14 airportdb@booking@0.csv
-rw-r--r--   1 shreyan  staff   69387753 Jul  6 22:13 airportdb@booking@1.csv
-rw-r--r--   1 shreyan  staff   73451383 Jul  6 22:15 airportdb@booking@10.csv
-rw-r--r--   1 shreyan  staff   73430082 Jul  6 22:14 airportdb@booking@11.csv
-rw-r--r--   1 shreyan  staff   73327619 Jul  6 22:15 airportdb@booking@12.csv
-rw-r--r--   1 shreyan  staff   73390040 Jul  6 22:15 airportdb@booking@13.csv
-rw-r--r--   1 shreyan  staff   73395914 Jul  6 22:15 airportdb@booking@14.csv
-rw-r--r--   1 shreyan  staff   73419293 Jul  6 22:15 airportdb@booking@1

In [22]:
from pyspark.sql import SparkSession

# Setting up Spark session with necessary configurations
spark = SparkSession \
    .builder \
    .appName("Parquetfy") \
    .config("spark.hadoop.fs.AbstractFileSystem.hdfs.impl", "org.apache.hadoop.hdfs.DistributedFileSystem") \
    .getOrCreate()

# Setting log level to ERROR to reduce log verbosity
spark.sparkContext.setLogLevel("ERROR")

# Reading CSV file with header and schema inference
df = spark.read.option("header", "true").option("inferSchema", "true").csv("csv_folder/airportdb@weatherdata@0.csv")

# Displaying the content of the DataFrame
df.show()

# Writing DataFrame to Parquet format
df.write.parquet("parquets/airportdb@weatherdata@0.parquet")


+----------------------------------------------------------+
|2005-01-01|00:00:00|1|0.0|99.0|1024.00|4.00|Schneefall|254|
+----------------------------------------------------------+
|                                      2005-01-01|00:00:...|
|                                      2005-01-01|00:00:...|
|                                      2005-01-01|00:00:...|
|                                      2005-01-01|00:05:...|
|                                      2005-01-01|00:05:...|
|                                      2005-01-01|00:05:...|
|                                      2005-01-01|00:05:...|
|                                      2005-01-01|00:10:...|
|                                      2005-01-01|00:10:...|
|                                      2005-01-01|00:10:...|
|                                      2005-01-01|00:10:...|
|                                      2005-01-01|00:15:...|
|                                      2005-01-01|00:15:...|
|                       

# Load Data to SQL

In [44]:
import json
import os
import re
import mysql

In [46]:
with open ("secrets.json", 'rb') as file:
    sql_credentials = json.load(file)

username = sql_credentials['username']
password = sql_credentials['password']
hostname = sql_credentials['ip']

In [49]:
# Define the directory where the CSV files are located
csv_directory = "csv_folder"

# Initialize a MySQL connection
connection = mysql.connector.connect(
    user=username
    , password=password
    , host=hostname
    , database='airportdb'
    , allow_local_infile=True
)
cursor = connection.cursor()

# Function to extract table name from filename (excluding numbers)
def extract_table_name(filename):
    table_name = os.path.splitext(filename)[0]  # Remove file extension
    table_name = re.sub(r'\d', '', table_name)  # Remove numbers
    return table_name

# Iterate through CSV files in the directory
for root, _, files in os.walk(csv_directory):
    for file in files:
        if file.endswith(".csv"):

            # Remove '@', '0' and 'airportdb' from the file name
            new_file_name = os.path.basename(file).replace('@', '').replace('0', '').replace('airportdb', '')
            
            # Extract the table name from the file name
            table_name = extract_table_name(new_file_name)

            # Construct the full path to the CSV file
            csv_file_path = os.path.join(root, file)

            # Replace backslashes with forward slashes for Windows paths
            # csv_file_path = tsv_file_path.replace("\\", "/")

            # Define the SQL query to load data from the CSV file
            load_data_query = f"""
                LOAD DATA LOCAL INFILE '{csv_file_path}' 
                INTO TABLE {table_name} 
                FIELDS TERMINATED BY '|' 
                LINES TERMINATED BY '\n';
            """

            # Execute the SQL query to load data
            try:
                cursor.execute(load_data_query)
                print(f"Loaded data from '{csv_file_path}' into table '{table_name}'")
            except mysql.connector.Error as err:
                print(f"Error loading data from '{csv_file_path}' into table '{table_name}': {err}")

# Commit changes and close the connection
connection.commit()
connection.close()

print("Data loading completed.")

Loaded data from 'csv_folder/airportdb@passengerdetails@@0.csv' into table 'passengerdetails'
Loaded data from 'csv_folder/airportdb@weatherdata@0.csv' into table 'weatherdata'
Loaded data from 'csv_folder/airportdb@weatherdata@@1.csv' into table 'weatherdata'
Loaded data from 'csv_folder/airportdb@airport_geo@@0.csv' into table 'airport_geo'
Loaded data from 'csv_folder/airportdb@booking@14.csv' into table 'booking'
Loaded data from 'csv_folder/airportdb@airplane_type@@0.csv' into table 'airplane_type'
Loaded data from 'csv_folder/airportdb@booking@15.csv' into table 'booking'
Loaded data from 'csv_folder/airportdb@booking@@24.csv' into table 'booking'
Loaded data from 'csv_folder/airportdb@booking@17.csv' into table 'booking'
Loaded data from 'csv_folder/airportdb@booking@16.csv' into table 'booking'
Loaded data from 'csv_folder/airportdb@booking@12.csv' into table 'booking'
Loaded data from 'csv_folder/airportdb@booking@13.csv' into table 'booking'
Loaded data from 'csv_folder/airpo